In [1]:
def ownershipFileReader():
    '''
    Reads and stores data in ownership.txt file 

    Returns:
        dictionary: Indexed by owner ID where each value is a dictionary containing 'Name' and 'Established'
    '''

    with open('ownership.txt', 'r') as file:
        fileData = file.read()

    # Dictionary to store the comapny name and its year established indexed by owner ID
    fileInfo = {}

    # Split file content based on newline token and iterate through each line (except first) storing the data
    fileLines = fileData.splitlines()[1:]
    for line in fileLines:
        ownerId, companyName, yearEstablished = line.split(',')
        fileInfo[int(ownerId)] = {
            'Name': companyName,
            'Established': yearEstablished
        }
    return fileInfo


In [2]:
def priceIndexFileReader():
    '''
    Reads and stores data in price_index.txt file 

    Returns:
        dictionary: Indexed by locality where each value is 'Avg Price Per sqft'
    '''

    with open('price_index.txt', 'r') as file:
        fileData = file.read()

    # Dictionary to store the avg price per sqft indexed by locality
    fileInfo = {}

    # Split file content based on newline token and iterate through each line (except first) storing the data
    fileLines = fileData.splitlines()[1:]
    for line in fileLines:
        locality, avgPricePerSqft = line.split(',')
        fileInfo[locality] = float(avgPricePerSqft)

    return fileInfo
    

In [3]:
def propertyFileReader():
    '''
    Reads and stores data in properties.txt file 

    Returns:
        dictionary: Indexed by locality where each value is a list containing dictionaries of 'OwnerID', 'Rooms' and 'Area'
    '''

    with open('properties.txt', 'r') as file:
        fileData = file.read()

    # Dictionary to store the location, rooms, and area of a property indexed by owner ID
    fileInfo = {}

    # Split file content based on newline token and iterate through each line (except first) storing the data
    fileLines = fileData.splitlines()[1:]
    for line in fileLines:
        locality, rooms, area, ownerId = line.split(',')
        if locality in fileInfo:
            fileInfo[locality].append({
                'OwnerID': int(ownerId),
                'Rooms': int(rooms),
                'Area': float(area)
            })
        else:
            fileInfo[locality] = [{
                'OwnerID': int(ownerId),
                'Rooms': int(rooms),
                'Area': float(area)
            }]
        
    return fileInfo

In [4]:
# Average Area - For each locality, calculate the average area of properties listed. 

def averageArea(propertyInfo):
    '''
    Calculates the average area of properties listed

    Args:
        propertyInfo (dictionary) : Dictionary indexed by locality where each value is a list containing dictionaries of 'OwnerID', 'Rooms' and 'Area'
    
    Returns:
        dictionary : Dictionary indexed by locality where each value is the corresponding average area of properties in the location
    '''

    avgArea = {}

    # Iterate through propertyInfo to calculate total area for each locality
    for locality, propertyData in propertyInfo.items():
        localityArea = 0
        # Iterate through each location's properties to find total area
        for propertyItem in propertyData:
            localityArea += propertyItem['Area']
        avgArea[locality] = localityArea / len(propertyData)

    return avgArea
    

In [5]:
# Average Price - For each area, calculate the average price of properties listed.

def averagePrice(avgArea, localityInfo):
    '''
    Calculates the average price of properties listed

    Args:
        avgArea (dictionary) : Dictionary indexed by locality where each value is the average property area of the location
        localityInfo (dictionary) : Dictionary indexed by locality where each value is 'Avg Price Per sqft'
        
    Returns:
        dictionary : Dictionary indexed by locality where each value is the corresponding average price of properties in the location
    '''

    avgPrice = {}

    # Iterate through localityInfo to calculate average price
    for localityKey, localityData in localityInfo.items():
        # Average Price = AvgArea * Price of each square foot
        avgPrice[localityKey] = avgArea[localityKey] * localityData

    return avgPrice
    

In [6]:
# Num of available properties - For each locality, count the number of available properties listed.

def numOfAvailableProperties(propertyInfo):
    '''
    Calculates the average area of properties listed

    Args:
        propertyInfo (dictionary) : Dictionary indexed by locality where each value is a dictionary containing 'OwnerID', 'Rooms' and 'Area'
        
    Returns:
        dictionary : Dictionary indexed by locality where each value is the corresponding number of available properties in the location
    '''
    availableProperties = {}
    
    # Iterate through propertyInfo to count available properties
    for locality, propertyData in propertyInfo.items():
        count = 0
        # Iterate through each location's properties to find total area
        for propertyItem in propertyData:
            if propertyItem['OwnerID'] == 0:
                count += 1
        availableProperties[locality] = count

    return availableProperties
    

In [7]:
def main():
    companyData = ownershipFileReader()
    localityData = priceIndexFileReader()
    propertyData = propertyFileReader()

    averageAreaData = averageArea(propertyData)
    averagePriceData = averagePrice(averageAreaData, localityData)
    availableProperties = numOfAvailableProperties(propertyData)

    # Sorting the locations based on number of available properties they have
    availableProperties = dict(sorted(
        availableProperties.items(), 
        key=lambda items: (-items[1], items[0]),
    ))
    
    # Line separator for the table
    separator = f'+{'-' * 15}+{'-' * 15}+{'-' * 15}+{'-' * 11}+'
    
    print(separator)

    # Printing the formatted title row for the table
    print(f'| {'Location':<14}|{'Average Area':^15}|{'Average Price':^15}|{'Available':^11}|')
    
    print(separator)

    # Iterate through each location, printing its corressponding information with formatting
    for locality in availableProperties.keys():
        location = f' {locality:<14}'
        avgArea = f'{averageAreaData[locality]:,.2f} sqft'.center(15)
        avgPrice = f' $ {averagePriceData[locality]:>11,.2f} '
        availability = f'{availableProperties[locality]:>10} '
        print(f'|{location}|{avgArea}|{avgPrice}|{availability}|')
    
    print(separator)
    
if __name__ == '__main__':
    main()

+---------------+---------------+---------------+-----------+
| Location      | Average Area  | Average Price | Available |
+---------------+---------------+---------------+-----------+
| Bonnie Doon   | 1,473.92 sqft | $  505,376.79 |         6 |
| Clareview     | 1,230.12 sqft | $  452,684.16 |         6 |
| Downtown      | 1,436.14 sqft | $  586,575.18 |         6 |
| McKernan      | 1,367.38 sqft | $  560,626.48 |         4 |
| Mill Woods    | 1,450.75 sqft | $  482,257.75 |         4 |
| Century Park  | 1,442.28 sqft | $  490,376.02 |         2 |
| Southgate     | 1,642.81 sqft | $  605,456.01 |         2 |
| Strathcona    | 1,447.04 sqft | $  549,875.98 |         2 |
+---------------+---------------+---------------+-----------+
